In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import time
import torch
from torch_geometric.loader import DataLoader
import argparse
import yaml
from torch_geometric.nn import radius
from torch.optim import Adam
import torch.autograd.profiler as profiler
from torch.profiler import profile, record_function, ProfilerActivity

from rbfConv import RbfConv
# from dataset import compressedFluidDataset, prepareData

import inspect
import re
def debugPrint(x):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} [{}] = {}".format(r,type(x).__name__, x))
%matplotlib notebook
import copy

import time
import torch
from torch_geometric.loader import DataLoader
from tqdm.notebook import trange, tqdm
import argparse
import yaml
from torch_geometric.nn import radius
from torch.optim import Adam
import torch.autograd.profiler as profiler
from torch.profiler import profile, record_function, ProfilerActivity

from rbfConv import RbfConv
from dataset import compressedFluidDataset, prepareData

import inspect
import re
def debugPrint(x):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} [{}] = {}".format(r,type(x).__name__, x))


import tomli
from scipy.optimize import minimize
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt

seed = 0


import random 
import numpy as np
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
# print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('running on: ', device)
torch.set_num_threads(1)

from joblib import Parallel, delayed

from cutlass import *
from rbfConv import *
from tqdm.notebook import tqdm

from datautils import *
# from sphUtils import *
from lossFunctions import *

In [3]:
torch.cuda.is_available()

False

In [4]:


"""Functions for reading the compressed training/validation data records"""
import os
import sys
import numpy as np
from glob import glob
import dataflow
import numpy as np
import zstandard as zstd
import msgpack
import msgpack_numpy
msgpack_numpy.patch()


class PhysicsSimDataFlow(dataflow.RNGDataFlow):
    """Data flow for msgpacks generated from SplishSplash simulations.
    """

    def __init__(self, files, random_rotation=False, shuffle=False, window=2):
        if not len(files):
            raise Exception("List of files must not be empty")
        if window < 1:
            raise Exception("window must be >=1 but is {}".format(window))
        self.files = files
        self.random_rotation = random_rotation
        self.shuffle = shuffle
        self.window = window

    def __iter__(self):
        decompressor = zstd.ZstdDecompressor()
        files_idxs = np.arange(len(self.files))
        if self.shuffle:
            self.rng.shuffle(files_idxs)

        for file_i in files_idxs:
            # read all data from file
            with open(self.files[file_i], 'rb') as f:
                data = msgpack.unpackb(decompressor.decompress(f.read()),
                                       raw=False)

            data_idxs = np.arange(len(data) - self.window + 1)
            if self.shuffle:
                self.rng.shuffle(data_idxs)

            # get box from first item. The box is valid for the whole file
            box = data[0]['box']
            box_normals = data[0]['box_normals']

            for data_i in data_idxs:

                if self.random_rotation:
                    angle_rad = self.rng.uniform(0, 2 * np.pi)
                    s = np.sin(angle_rad)
                    c = np.cos(angle_rad)
                    rand_R = np.array([c, 0, s, 0, 1, 0, -s, 0, c],
                                      dtype=np.float32).reshape((3, 3))

                if self.random_rotation:
                    sample = {
                        'box': np.matmul(box, rand_R),
                        'box_normals': np.matmul(box_normals, rand_R)
                    }
                else:
                    sample = {'box': box, 'box_normals': box_normals}

                for time_i in range(self.window):

                    item = data[data_i + time_i]

                    for k in ('pos', 'vel'):
                        if self.random_rotation:
                            sample[k + str(time_i)] = np.matmul(item[k], rand_R)
                        else:
                            sample[k + str(time_i)] = item[k]

                    for k in ('m', 'viscosity', 'frame_id', 'scene_id'):
                        sample[k + str(time_i)] = item[k]

                yield sample


def read_data_val(files, **kwargs):
    return read_data(files=files,
                     batch_size=1,
                     repeat=False,
                     shuffle_buffer=None,
                     num_workers=1,
                     **kwargs)


def read_data_train(files, batch_size, random_rotation=True, **kwargs):
    return read_data(files=files,
                     batch_size=batch_size,
                     random_rotation=random_rotation,
                     repeat=True,
                     shuffle_buffer=512,
                     **kwargs)


def read_data(files=None,
              batch_size=1,
              window=2,
              random_rotation=False,
              repeat=False,
              shuffle_buffer=None,
              num_workers=1,
              cache_data=False):
    print(files[0:20], '...' if len(files) > 20 else '')

    # caching makes only sense if the data is finite
    if cache_data:
        if repeat == True:
            raise Exception("repeat must be False if cache_data==True")
        if random_rotation == True:
            raise Exception("random_rotation must be False if cache_data==True")
        if num_workers != 1:
            raise Exception("num_workers must be 1 if cache_data==True")

    df = PhysicsSimDataFlow(
        files=files,
        random_rotation=random_rotation,
        shuffle=True if shuffle_buffer else False,
        window=window,
    )

    if repeat:
        df = dataflow.RepeatedData(df, -1)

    if shuffle_buffer:
        df = dataflow.LocallyShuffleData(df, shuffle_buffer)

    if num_workers > 1:
        df = dataflow.MultiProcessRunnerZMQ(df, num_proc=num_workers)

    df = dataflow.BatchData(df, batch_size=batch_size, use_list=True)

    if cache_data:
        df = dataflow.CacheData(df)

    df.reset_state()
    return df

/home/winchenbach/anaconda3/envs/torch_env/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/winchenbach/anaconda3/envs/torch_env/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/home/winchenbach/anaconda3/envs/torch_env/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/home/winchenbach/anaconda3/envs/torch_env/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:28: DeprecationWarning: HAMMING is deprecated and will be removed in Pillow 10 (2023-0

In [5]:
os.path.join(os.path.expanduser('~/dev/datasets/WBCSPH2D'), 'train', '*.zst')

'/home/winchenbach/dev/datasets/WBCSPH2D/train/*.zst'

In [4]:

train_files = sorted(
    glob(os.path.join(os.path.expanduser('~/dev/datasets/WBCSPH2D'), 'train', '*.zst')))

dataset = read_data_train(files=train_files,
                          batch_size=16,
                          window=3,
                          num_workers=1
                          )

df = PhysicsSimDataFlow(
    files=train_files,
    random_rotation=False,
    shuffle=False,
    window=3,
)

for frame in df:
#     print(frame.attrs.keys())
    print(frame.keys())
    print(frame['box'].shape)
    print(frame['box_normals'].shape)
    print(frame['pos0'].shape, frame['pos0'])
    print(frame['vel0'].shape, frame['vel0'])
    print(frame['m0'].shape, frame['m0'])
    print(frame['viscosity0'].shape, frame['viscosity0'])
    print(frame['frame_id0'])
    print(frame['scene_id0'])
    print(frame['pos1'].shape)
    print(frame['vel1'].shape)
    print(frame['m1'].shape)
    print(frame['viscosity1'].shape)
    print(frame['frame_id1'])
    print(frame['scene_id1'])
    print(frame['pos2'].shape)
    print(frame['vel2'].shape)
    print(frame['m2'].shape)
    print(frame['viscosity2'].shape)
    print(frame['frame_id2'])
    print(frame['scene_id2'])
    break

['/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_00.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_01.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_02.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_03.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_04.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_05.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_06.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0000_07.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_00.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_01.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_02.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_03.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim_0002_04.msgpack.zst', '/home/winchenbach/dev/datasets/WBCSPH2D/train/sim

In [61]:

decompressor = zstd.ZstdDecompressor()
files_idxs = np.arange(len(train_files))

for file_i in files_idxs:
    # read all data from file
    with open(train_files[file_i], 'rb') as f:
        data = msgpack.unpackb(decompressor.decompress(f.read()),
                               raw=False)

    data_idxs = np.arange(len(data))

    # get box from first item. The box is valid for the whole file
    box = data[0]['box']
    box_normals = data[0]['box_normals']
    
    print(data[0].keys())
    print(data[0])
    break
    

dict_keys(['box', 'box_normals', 'frame_id', 'scene_id', 'num_rigid_bodies', 'pos', 'vel', 'm', 'viscosity', 'force', 'grav'])
{'box': array([[0.0025, 0.0025, 0.    ],
       [0.0075, 0.0025, 0.    ],
       [0.0025, 0.0075, 0.    ],
       ...,
       [0.9975, 0.9925, 0.    ],
       [0.9925, 0.9975, 0.    ],
       [0.9975, 0.9975, 0.    ]]), 'box_normals': array([[-0.70710678, -0.70710678, -0.        ],
       [-0.07741782, -0.99699874, -0.        ],
       [-0.99699874, -0.07741782, -0.        ],
       ...,
       [ 0.997402  ,  0.0720364 , -0.        ],
       [ 0.0720364 ,  0.997402  , -0.        ],
       [ 0.70710678,  0.70710678, -0.        ]]), 'frame_id': 0, 'scene_id': 'sim_0000', 'num_rigid_bodies': 0, 'pos': array([[0.5825, 0.0425, 0.    ],
       [0.5875, 0.0425, 0.    ],
       [0.5825, 0.0475, 0.    ],
       ...,
       [0.1575, 0.9525, 0.    ],
       [0.1525, 0.9575, 0.    ],
       [0.1575, 0.9575, 0.    ]]), 'vel': array([[0., 0., 0.],
       [0., 0., 0.],
      

In [ ]:
basePath = '../export'
basePath = os.path.expanduser(basePath)

simulationFiles = [basePath + '/' + f for f in os.listdir(basePath) if f.endswith('.hdf5')]



In [6]:
dist = 16
inFile = h5py.File(simulationFiles[0], 'r')
frameCount = int(len(inFile['simulationExport'].keys()))

sublists = np.array_split(np.arange(frameCount // dist), 16 / dist)

print(frameCount)

2000


In [7]:
box = np.float32(np.hstack((inFile['boundaryInformation']['boundaryPosition'], np.zeros((inFile['boundaryInformation']['boundaryPosition'].shape[0],1)))))
box_normals = np.float32(np.hstack((inFile['boundaryInformation']['boundaryNormals'], np.zeros((inFile['boundaryInformation']['boundaryNormals'].shape[0],1)))))

In [8]:
prefix = simulationFiles[0].split('/')[-1].split('.')[0]

s = 0
exportPath = '%s - %2d - %2d [%2d].msgpack.zst' % (prefix, dist, offset, s)

print(prefix)
print(exportPath)
if not os.path.exists('./convertedData/'):
    os.makedirs('./convertedData/')

NameError: name 'offset' is not defined

In [ ]:

def create_compressed_msgpack(data, outfilepath):
    import zstandard as zstd
    import msgpack
    import msgpack_numpy
    msgpack_numpy.patch()

    compressor = zstd.ZstdCompressor(level=0)
    with open(outfilepath, 'wb') as f:
#         print('writing', outfilepath)
        f.write(compressor.compress(msgpack.packb(data, use_bin_type=True)))

In [ ]:
from tqdm.notebook import tqdm

In [9]:

data = []
offset = 0
for offset in tqdm(range(dist)):
    for s, sl in enumerate(tqdm(sublists, leave = False)):
        outfilepath = './convertedData/%s - %02d - %02d [%02d].msgpack.zst' % (prefix, dist, offset, s)
#         print(outfilepath)
        data = []
#         print(sl)
        for si, ii in enumerate(sl):
            i = ii * dist + offset
            if i > len(inFile['simulationExport'].keys()):
                break
            frame = {}
            if si == 0:
#                 print(si)
                frame['box'] = box
                frame['box_normals'] = box_normals
    #         print(i)
        #     print(inFile['boundaryInformation'].keys())

        #     print(torch.from_numpy(inFile['boundaryInformation']['boundaryNormals'][:]))
            frameGroup = inFile['simulationExport']['%05d' % i]
            frame['pos']       = np.float32(np.hstack((frameGroup['fluidPosition'], np.zeros((frameGroup['fluidArea'].shape[0],1)))))
            frame['vel']       = np.float32(np.hstack((frameGroup['fluidVelocity'], np.zeros((frameGroup['fluidArea'].shape[0],1)))))
            frame['m']         = np.float32(np.expand_dims(frameGroup['fluidArea'][:], axis=1) * 998)
            frame['viscosity'] = np.float32(np.expand_dims(frameGroup['fluidArea'], axis=1) * 0)
            frame['frame_id']  = ii
            frame['scene_id']  = '%s - %2d - %2d' % (prefix, dist, offset)
            data.append(frame)
#         print(len(data))
    #         break
    #     print(data)
        create_compressed_msgpack(data, outfilepath)

  0%|                                                                                                                                    | 0/16 [00:00<?, ?it/s]


NameError: name 'create_compressed_msgpack' is not defined

In [10]:

train_files = sorted(
    glob(os.path.join('./convertedData/', 'train', '*.zst')))

dataset = read_data_train(files=train_files,
                          batch_size=16,
                          window=3,
                          num_workers=1
                          )

df = PhysicsSimDataFlow(
    files=train_files,
    random_rotation=False,
    shuffle=False,
    window=3,
)

for frame in df:
#     print(frame.attrs.keys())
    print(frame.keys())
    print(frame['box'].shape, frame['box'].dtype)
    print(frame['box_normals'].shape, frame['box_normals'].dtype)
    print(frame['pos0'].shape, frame['pos0'].dtype)
    print(frame['vel0'].shape, frame['vel0'].dtype)
    print(frame['m0'].shape, frame['m0'].dtype)
    print(frame['viscosity0'].shape, frame['viscosity0'].dtype)
    print(frame['frame_id0'])
    print(frame['scene_id0'])
    print(frame['pos1'].shape)
    print(frame['vel1'].shape)
    print(frame['m1'].shape)
    print(frame['viscosity1'].shape)
    print(frame['frame_id1'])
    print(frame['scene_id1'])
    print(frame['pos2'].shape)
    print(frame['vel2'].shape)
    print(frame['m2'].shape)
    print(frame['viscosity2'].shape)
    print(frame['frame_id2'])
    print(frame['scene_id2'])
    break

['./convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 00 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 01 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 02 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 03 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 04 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 05 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 06 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 07 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 08 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 09 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 10 [00].msgpack.zst', './convertedData/train/dambreak - 2023-02-22_10-49-21 - 16 - 11 [00].msgpack.zst', './

In [11]:
basePath = '~/data/'
basePath = os.path.expanduser(basePath)

simulationFiles = [basePath + '/' + f for f in os.listdir(basePath) if f.endswith('.zst')]
# for i, c in enumerate(simulationFiles):
#     print(i ,c)
#     
simulationFiles  = [simulationFiles[0]]

training = []
validation = []
testing = []


for s in simulationFiles:    
    _, train, valid, test = splitFileZSTD(s, split = True, limitRollOut = False, skip = 0, cutoff = 1800, distance = 1)
    training.append((s,train))
    validation.append((s,valid))
    testing.append((s,test))

In [12]:

def splitFileZSTD(s, skip = 32, cutoff = 300, chunked = True, maxRollOut = 8, split = True, trainValidationSplit = 0.8, testSplit = 0.1, limitRollOut = False, distance = 1):
    with open(simulationFiles[0], 'rb') as f:
        data = msgpack.unpackb(decompressor.decompress(f.read()),
                               raw=False)
        frameCount = len(data)
#     inFile = h5py.File(s, 'r')
#     frameCount = int(len(inFile['simulationExport'].keys()) -1) // distance # adjust for bptcls
#     inFile.close()
    if cutoff > 0:
        frameCount = min(cutoff+skip, frameCount)
    actualCount = frameCount - 1 - skip
    
    if cutoff < 0:
        actualCount = frameCount + cutoff
    
    if not split:
        print(frameCount, cutoff, actualCount)
        training, _, counter = getSamples(actualCount, maxRollOut = maxRollOut, chunked = chunked, trainValidationSplit = 1.)
        return s, training + skip, counter
    
    testIndex = frameCount - 1 - int(actualCount * testSplit)
    testSamples = frameCount - 1 - testIndex
    
    # print(frameCount, cutoff, testSamples)
    testingIndices, _, testingCounter = getSamples(testSamples, maxRollOut = maxRollOut, chunked = chunked, trainValidationSplit = 1.)
    testingIndices = testingIndices * distance + testIndex
    
    # print(frameCount, cutoff, testIndex - skip)
    trainingIndices, validationIndices, trainValidationCounter = getSamples(testIndex - skip, maxRollOut = maxRollOut, chunked = chunked, trainValidationSplit = trainValidationSplit, limitRollOut = limitRollOut)
    trainingCounter = trainValidationCounter[trainingIndices]
    validationCounter = -trainValidationCounter[validationIndices]
    
    trainingIndices = trainingIndices * distance + skip
    validationIndices = validationIndices * distance + skip
    
    # print(trainingIndices.shape[0])
    # print(validationIndices.shape[0])
    # print(testingIndices.shape[0])
    
    return s, (trainingIndices, trainingCounter), (validationIndices, validationCounter), (testingIndices, testingCounter)
    
    
def loadFrameZSTD(filename, frame, frameOffsets = [1], frameDistance = 1):
    with open(simulationFiles[0], 'rb') as f:
        data = msgpack.unpackb(decompressor.decompress(f.read()),
                               raw=False)
        frameCount = len(data)
        debugPrint(data[0]['box'])
        
#     inFile = h5py.File(filename)
#     inGrp = inFile['simulationExport']['%05d' % frame]
#     debugPrint(inFile.attrs.keys())
        attributes = {
         'support': 0.0025,
         'targetNeighbors': 20,
         'restDensity': 4,
         'dt': 0.0025,
         'time': frame * 0.0025,
         'radius': 0.0025,
         'area': 0.0025**2 * np.pi,
        }
    #     debugPrint(inGrp.attrs['timestep'])

    #     support = inFile.attrs['restDensity']
    #     targetNeighbors = inFile.attrs['targetNeighbors']
    #     restDensity = inFile.attrs['restDensity']
    #     dt = inFile.attrs['initialDt']

        gravity =  torch.zeros_like(torch.from_numpy(data[frame]['vel'][:]).type(torch.float32))
        gravity[:,0] = data[frame]['grav'][0]
        gravity[:,1] = data[frame]['grav'][1]

        inputData = {
            'fluidPosition': torch.from_numpy(data[frame]['pos'][:,:2]).type(torch.float32),
            'fluidVelocity': torch.from_numpy(data[frame]['vel'][:,:2]).type(torch.float32),
            'fluidArea' : torch.from_numpy(data[frame]['m']).type(torch.float32),
    #         'fluidDensity' : torch.from_numpy(inGrp['fluidDensity'][:]).type(torch.float32),
    #         'fluidSupport' : torch.from_numpy(inGrp['fluidSupport'][:]).type(torch.float32),
            'fluidGravity': gravity,
    #         'fluidGravity' : torch.from_numpy(inGrp['fluidGravity'][:]).type(torch.float32) if 'fluidGravity' not in inFile.attrs else torch.from_numpy(inFile.attrs['fluidGravity']).type(torch.float32) * torch.ones(inGrp['fluidDensity'][:].shape[0])[:,None],
            'boundaryPosition': torch.from_numpy(data[0]['box'][:,:2]).type(torch.float32),
            'boundaryNormal': torch.from_numpy(data[0]['box_normals'][:,:2]).type(torch.float32),
    #         'boundaryArea': torch.from_numpy(inFile[frame]['boundaryArea'][:]).type(torch.float32),
    #         'boundaryVelocity': torch.from_numpy(inFile['boundaryInformation']['boundaryVelocity'][:]).type(torch.float32)
        }

        groundTruthData = []
        for i in frameOffsets:
#             gtGrp = inFile['simulationExport']['%05d' % (frame + i * frameDistance)]
    #         debugPrint((frame + i * frameDistance))
    #         debugPrint(gtGrp.attrs['timestep'])
            gtData = {
                'fluidPosition'    : torch.from_numpy(data[frame + i * frameDistance]['pos'][:,:2]).type(torch.float32),
                'fluidVelocity'    : torch.from_numpy(data[frame + i * frameDistance]['vel'][:,:2]).type(torch.float32),
                'fluidDensity'     : torch.from_numpy(data[frame + i * frameDistance]['m'][:]).type(torch.float32)
        #         'fluidPressure'    : torch.from_numpy(gtGrp['fluidPressure'][:]),
        #         'boundaryDensity'  : torch.from_numpy(gtGrp['fluidDensity'][:]),
        #         'boundaryPressure' : torch.from_numpy(gtGrp['fluidPressure'][:]),
            }
        
            groundTruthData.append(torch.hstack((gtData['fluidPosition'].type(torch.float32), gtData['fluidVelocity'], gtData['fluidDensity'])))


        inFile.close()

        return attributes, inputData, groundTruthData


In [13]:

decompressor = zstd.ZstdDecompressor()
with open(simulationFiles[0], 'rb') as f:
    data = msgpack.unpackb(decompressor.decompress(f.read()),
                           raw=False)
    print(len(data))

400


In [14]:
splitFileZSTD(simulationFiles[0], split = False, cutoff = -3, skip = 0)

400 -3 397


('/home/winchenbach/data//sim_0000_01.msgpack.zst',
 array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,

In [15]:

with open(simulationFiles[0], 'rb') as f:
    data = msgpack.unpackb(decompressor.decompress(f.read()),
                           raw=False)
    print(data[0])

{'box': array([[0.0025, 0.0025, 0.    ],
       [0.0075, 0.0025, 0.    ],
       [0.0025, 0.0075, 0.    ],
       ...,
       [0.9975, 0.9925, 0.    ],
       [0.9925, 0.9975, 0.    ],
       [0.9975, 0.9975, 0.    ]]), 'box_normals': array([[-0.70710678, -0.70710678, -0.        ],
       [-0.07741782, -0.99699874, -0.        ],
       [-0.99699874, -0.07741782, -0.        ],
       ...,
       [ 0.997402  ,  0.0720364 , -0.        ],
       [ 0.0720364 ,  0.997402  , -0.        ],
       [ 0.70710678,  0.70710678, -0.        ]]), 'frame_id': 400, 'scene_id': 'sim_0000', 'num_rigid_bodies': 0, 'pos': array([[0.2413765 , 0.10861348, 0.        ],
       [0.44395267, 0.12779327, 0.        ],
       [0.33709118, 0.24703171, 0.        ],
       ...,
       [0.14774105, 0.94179474, 0.        ],
       [0.13117629, 0.93664238, 0.        ],
       [0.11658319, 0.94571251, 0.        ]]), 'vel': array([[ 0.35902485, -0.1798736 ,  0.        ],
       [ 0.13369633, -0.9626886 ,  0.        ],
     

In [30]:

from tqdm.notebook import trange, tqdm

In [16]:
# loadFrameZSTD(simulationFiles[0],0,np.arange(2) + 1, frameDistance = 1)

In [75]:

simulationFiles = sorted(
    glob(os.path.join(os.path.expanduser('~/dev/datasets/WBCSPH2D'), 'train', '*.zst')))


In [76]:
# print(simulationFiles)
splitNames = [s.split('/')[-1].split('.')[0].split('_') for s in simulationFiles]
simulations = sorted(list(set([s[1] for s in splitNames])))

collected = {}
for prefix, sim, frame in splitNames:
    if sim not in collected:
        collected[sim] = []
    collected[sim].append('~/dev/datasets/WBCSPH2D/train/' + prefix + '_' + sim + '_' + frame + '.msgpack.zst')

for c in collected:
    collected[c] = sorted(collected[c])
    
# print(collected)

outFile.close()
for c in tqdm(collected):
#     print(c, collected[c])
    decompressor = zstd.ZstdDecompressor()
    
    exportPath = os.path.expanduser('~/dev/datasets/WBCSPH2D/train/sim_' + c + '.hdf5')
#     print(exportPath)
    outFile = h5py.File(exportPath,'w')
    
    with open(os.path.expanduser(collected[c][0]), 'rb') as f:
        data = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)
    outFile.attrs['support'] = 0.0025
    outFile.attrs['targetNeighbors'] = 20
    outFile.attrs['restDensity'] = 4
    outFile.attrs['radius'] = 0.0025
    outFile.attrs['area'] = 0.0025**2 * np.pi
    outFile.attrs['fluidGravity'] = data[0]['grav'][:2]
    
    boundaryGrp = outFile.create_group('boundaryInformation')

    boundaryGrp.create_dataset('boundaryPosition', data = data[0]['box'][:,:2])
    boundaryGrp.create_dataset('boundaryNormals', data = data[0]['box_normals'][:,:2])
    boundaryGrp.create_dataset('boundaryArea', data = np.ones_like(data[0]['box'][:,0]))
    boundaryGrp.create_dataset('boundaryVelocity', data = np.ones_like(data[0]['box']))
    
    simExport = outFile.create_group('simulationExport')
    
    ii = 0
    for l in tqdm(collected[c], leave = False):
        with open(os.path.expanduser(l), 'rb') as f:
            data = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)
    
        for i in tqdm(range(len(data)), leave = False):
            grp = simExport.create_group('%05d' % ii)

            grp.attrs['dt'] = 0.0025
            grp.attrs['time'] = ii * 0.0025

            grp.create_dataset('fluidPosition', data = data[i]['pos'][:,:2])
            grp.create_dataset('fluidVelocity', data = data[i]['vel'][:,:2])
            grp.create_dataset('fluidArea', data = data[i]['m'][:,0])

            grp.create_dataset('fluidDensity', data = np.ones_like(data[i]['m'][:,0]))
            grp.create_dataset('fluidSupport', data = np.ones_like(data[i]['m'][:,0]) * 0.0025)
            ii = ii + 1

    outFile.close()
#     break


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

In [34]:

def create_compressed_msgpack(data, outfilepath):
    import zstandard as zstd
    import msgpack
    import msgpack_numpy
    msgpack_numpy.patch()

    compressor = zstd.ZstdCompressor(level=0)
    with open(outfilepath, 'wb') as f:
#         print('writing', outfilepath)
        f.write(compressor.compress(msgpack.packb(data, use_bin_type=True)))

In [77]:

simulationFiles = sorted(
    glob(os.path.join(os.path.expanduser('~/dev/datasets/WBCSPH2Dc'), 'train', '*.hdf5')))
print(simulationFiles)

['/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0000.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0002.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0003.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0004.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0005.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0006.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0007.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0010.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0012.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0013.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0014.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0016.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0018.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0019.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2Dc/train/sim_0020.hdf5', '/home/winchenbach/dev/datasets/WBCSPH2

In [78]:
inFile.close()
for s in tqdm(simulationFiles):
    p = s.split('.')
    outPath = p[0] + '.msgpack.zst'
    data = []
    
    inFile = h5py.File(s)
    
#     print(inFile['simulationExport'].keys())
    keys = list(inFile['simulationExport'].keys())
    for i in range(0, len(inFile['simulationExport'].keys()), 8):
        frame = {}
        if i == 0:
            frame['box'] = np.float32(np.hstack((inFile['boundaryInformation']['boundaryPosition'][:], np.zeros((inFile['boundaryInformation']['boundaryPosition'][:].shape[0],1)))))
            frame['box_normals'] = np.float32(np.hstack((inFile['boundaryInformation']['boundaryNormals'][:], np.zeros((inFile['boundaryInformation']['boundaryNormals'][:].shape[0],1)))))
#         print(frame['box'].dtype)
        frame['pos'] = np.float32(np.hstack((inFile['simulationExport'][keys[i]]['fluidPosition'][:], np.zeros((inFile['simulationExport'][keys[i]]['fluidPosition'][:].shape[0],1)))))
        frame['vel'] = np.float32(np.hstack((inFile['simulationExport'][keys[i]]['fluidVelocity'][:], np.zeros((inFile['simulationExport'][keys[i]]['fluidVelocity'][:].shape[0],1)))))
        frame['m'] = np.float32(np.expand_dims((inFile['simulationExport'][keys[i]]['fluidArea'][:]), axis = 1))
        frame['viscosity'] = np.zeros_like(frame['m'])
        
        frame['frame_id'] = i
        frame['scene_id'] = s.split('/')[-1].split('.')[0]
        frame['num_rigid_bodies'] = 0
        frame['grav'] = np.hstack((inFile.attrs['fluidGravity'],0))
        
#         print(frame)
        data.append(frame)
#         break
    
#     print(len(data))
    inFile.close()
#     print(outPath)
    
    create_compressed_msgpack(data, outPath)
#     break

  0%|          | 0/35 [00:00<?, ?it/s]

In [79]:
torch.cuda.is_available()

True

In [ ]:

data = []
offset = 0
for offset in tqdm(range(dist)):
    for s, sl in enumerate(tqdm(sublists, leave = False)):
        outfilepath = './convertedData/%s - %02d - %02d [%02d].msgpack.zst' % (prefix, dist, offset, s)
#         print(outfilepath)
        data = []
#         print(sl)
        for si, ii in enumerate(sl):
            i = ii * dist + offset
            if i > len(inFile['simulationExport'].keys()):
                break
            frame = {}
            if si == 0:
#                 print(si)
                frame['box'] = box
                frame['box_normals'] = box_normals
    #         print(i)
        #     print(inFile['boundaryInformation'].keys())

        #     print(torch.from_numpy(inFile['boundaryInformation']['boundaryNormals'][:]))
            frameGroup = inFile['simulationExport']['%05d' % i]
            frame['pos']       = np.float32(np.hstack((frameGroup['fluidPosition'], np.zeros((frameGroup['fluidArea'].shape[0],1)))))
            frame['vel']       = np.float32(np.hstack((frameGroup['fluidVelocity'], np.zeros((frameGroup['fluidArea'].shape[0],1)))))
            frame['m']         = np.float32(np.expand_dims(frameGroup['fluidArea'][:], axis=1) * 998)
            frame['viscosity'] = np.float32(np.expand_dims(frameGroup['fluidArea'], axis=1) * 0)
            frame['frame_id']  = ii
            frame['scene_id']  = '%s - %2d - %2d' % (prefix, dist, offset)
            data.append(frame)
#         print(len(data))
    #         break
    #     print(data)
        create_compressed_msgpack(data, outfilepath)

In [19]:
def convert(s):
    decompressor = zstd.ZstdDecompressor()
    with open(s, 'rb') as f:
        data = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)
    exportPath = s.split('.')[0] + '.hdf5'
    outFile = h5py.File(exportPath,'w')
#     print(exportPath)

    outFile.attrs['support'] = 0.0025
    outFile.attrs['targetNeighbors'] = 20
    outFile.attrs['restDensity'] = 4
    outFile.attrs['radius'] = 0.0025
    outFile.attrs['area'] = 0.0025**2 * np.pi
    outFile.attrs['fluidGravity'] = data[0]['grav']

    boundaryGrp = outFile.create_group('boundaryInformation')

    boundaryGrp.create_dataset('boundaryPosition', data = data[0]['box'][:,:2])
    boundaryGrp.create_dataset('boundaryNormals', data = data[0]['box_normals'][:,:2])
    boundaryGrp.create_dataset('boundaryArea', data = np.ones_like(data[0]['box'][:,0]))
    boundaryGrp.create_dataset('boundaryVelocity', data = np.ones_like(data[0]['box']))
    
    simExport = outFile.create_group('simulationExport')

    for i in tqdm(range(len(data)), leave = False):
        grp = simExport.create_group('%05d' % i)
        
        grp.attrs['dt'] = 0.0025
        grp.attrs['time'] = i * 0.0025

        grp.create_dataset('fluidPosition', data = data[i]['pos'][:,:2])
        grp.create_dataset('fluidVelocity', data = data[i]['vel'][:,:2])
        grp.create_dataset('fluidArea', data = data[i]['m'][:,0])

        grp.create_dataset('fluidDensity', data = np.ones_like(data[i]['m'][:,0]))
        grp.create_dataset('fluidSupport', data = np.ones_like(data[i]['m'][:,0]) * 0.0025)

    #     break

    outFile.close()

In [20]:

from tqdm.notebook import trange, tqdm

In [ ]:
for s in tqdm(simulationFiles):
    convert(s)

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

...

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

In [66]:
outFile.close()

<HDF5 dataset "boundaryVelocity": shape (16057, 3), type "<f8">

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:

    with open(simulationFiles[0], 'rb') as f:
        data = msgpack.unpackb(decompressor.decompress(f.read()),
                               raw=False)
        frameCount = len(data)
        debugPrint(data[0]['box'])
        
#     inFile = h5py.File(filename)
#     inGrp = inFile['simulationExport']['%05d' % frame]
#     debugPrint(inFile.attrs.keys())
        attributes = {
         'support': 0.0025,
         'targetNeighbors': 20,
         'restDensity': 4,
         'dt': 0.0025,
         'time': frame * 0.0025,
         'radius': 0.0025,
         'area': 0.0025**2 * np.pi,
        }
    #     debugPrint(inGrp.attrs['timestep'])

    #     support = inFile.attrs['restDensity']
    #     targetNeighbors = inFile.attrs['targetNeighbors']
    #     restDensity = inFile.attrs['restDensity']
    #     dt = inFile.attrs['initialDt']

        gravity =  torch.zeros_like(torch.from_numpy(data[frame]['vel'][:]).type(torch.float32))
        gravity[:,0] = data[frame]['grav'][0]
        gravity[:,1] = data[frame]['grav'][1]

        inputData = {
            'fluidPosition': torch.from_numpy(data[frame]['pos'][:,:2]).type(torch.float32),
            'fluidVelocity': torch.from_numpy(data[frame]['vel'][:,:2]).type(torch.float32),
            'fluidArea' : torch.from_numpy(data[frame]['m']).type(torch.float32),
    #         'fluidDensity' : torch.from_numpy(inGrp['fluidDensity'][:]).type(torch.float32),
    #         'fluidSupport' : torch.from_numpy(inGrp['fluidSupport'][:]).type(torch.float32),
            'fluidGravity': gravity,
    #         'fluidGravity' : torch.from_numpy(inGrp['fluidGravity'][:]).type(torch.float32) if 'fluidGravity' not in inFile.attrs else torch.from_numpy(inFile.attrs['fluidGravity']).type(torch.float32) * torch.ones(inGrp['fluidDensity'][:].shape[0])[:,None],
            'boundaryPosition': torch.from_numpy(data[0]['box'][:,:2]).type(torch.float32),
            'boundaryNormal': torch.from_numpy(data[0]['box_normals'][:,:2]).type(torch.float32),
    #         'boundaryArea': torch.from_numpy(inFile[frame]['boundaryArea'][:]).type(torch.float32),
    #         'boundaryVelocity': torch.from_numpy(inFile['boundaryInformation']['boundaryVelocity'][:]).type(torch.float32)
        }

        groundTruthData = []
        for i in frameOffsets:
#             gtGrp = inFile['simulationExport']['%05d' % (frame + i * frameDistance)]
    #         debugPrint((frame + i * frameDistance))
    #         debugPrint(gtGrp.attrs['timestep'])
            gtData = {
                'fluidPosition'    : torch.from_numpy(data[frame + i * frameDistance]['pos'][:,:2]).type(torch.float32),
                'fluidVelocity'    : torch.from_numpy(data[frame + i * frameDistance]['vel'][:,:2]).type(torch.float32),
                'fluidDensity'     : torch.from_numpy(data[frame + i * frameDistance]['m'][:]).type(torch.float32)
        #         'fluidPressure'    : torch.from_numpy(gtGrp['fluidPressure'][:]),
        #         'boundaryDensity'  : torch.from_numpy(gtGrp['fluidDensity'][:]),
        #         'boundaryPressure' : torch.from_numpy(gtGrp['fluidPressure'][:]),
            }
        
            groundTruthData.append(torch.hstack((gtData['fluidPosition'].type(torch.float32), gtData['fluidVelocity'], gtData['fluidDensity'])))


        inFile.close()

        return attributes, inputData, groundTruthData